In [1]:
# Loading the Libraries
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os,glob

GEO_NAME_PATH = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/geo_name_test.csv'
geo_name = pd.read_csv(GEO_NAME_PATH)

In [2]:
geo_name['geo_level'] = geo_name['geo_level'].str.lower()
geo_name['parent_alt_geo_code'] = ""
province = geo_name[geo_name['geo_level'] == 'province']
census_division = geo_name[geo_name['geo_level'] == 'census division']
census_subdivision = geo_name[geo_name['geo_level'] == 'census subdivision']

In [3]:
province['parent_dguid'] = 'Canada'
census_division['parent_alt_geo_code'] = census_division['alt_geo_code'].astype(str).str[:2].astype(int)
census_subdivision['parent_alt_geo_code'] = census_subdivision['alt_geo_code'].astype(str).str[:4].astype(int)

/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_27004/4032030726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  province['parent_dguid'] = 'Canada'
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_27004/4032030726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_division['parent_alt_geo_code'] = census_division['alt_geo_code'].astype(str).str[:2].astype(int)
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_27004/4032030726.py:3: SettingWithCopyWarning: 
A value 

In [4]:
merged_census_division = census_division.merge(province, left_on='parent_alt_geo_code', right_on='alt_geo_code', how='left')
merged_census_subdivision = census_subdivision.merge(census_division, left_on='parent_alt_geo_code', right_on='alt_geo_code', how='left')
# census_division

In [5]:
merged_census_division = merged_census_division[['dguid_x', 'alt_geo_code_x', 'geo_level_x', 'geo_name_x', 'dguid_y']]
merged_census_subdivision = merged_census_subdivision[['dguid_x', 'alt_geo_code_x', 'geo_level_x', 'geo_name_x', 'dguid_y']]

In [6]:
census = pd.concat([merged_census_division, merged_census_subdivision])
census.rename({'dguid_x': 'dguid', 'alt_geo_code_x': 'alt_geo_code', 'geo_level_x': 'geo_level', 'geo_name_x': 'geo_name', 'dguid_y': 'parent_dguid'}, axis=1, inplace=True)


In [8]:
final = pd.concat([province, census])
# drop parent_alt_geo_code
final.drop(['parent_alt_geo_code'], axis=1, inplace=True)
final.to_csv('geo_name_final.csv', index=False)

In [9]:
# Local Connection
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + "root" + ':' + "Chuckie2020!" + '@' + "localhost" + '/' + "canada_vis" 

engine = create_engine(conn_string, connect_args=ssl_args)

with engine.begin() as connection:
    final.to_sql('geo_names', con=connection, if_exists='replace')